In [4]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [5]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W51`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W52`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W03`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W04`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W05`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W06`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W07`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W08`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W09`
"""

In [6]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [7]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
37328,2021-02-25 06:19:33+00:00,2021-02-25 06:19:46+00:00,Production,0001 of 0002,X4,None,None,KL15,Sun-SI Delivery,Thursday Morning,iXUsr_Tafai(10143),3,0.22,2021-09
37347,2021-02-25 06:16:33+00:00,2021-02-25 06:21:54+00:00,Production,0002 of 0011,E4,None,None,KL7,Sun-SI Delivery,Thursday Morning,iXUsr_Gwen(7292),3,5.35,2021-09
37348,2021-02-25 07:54:46+00:00,2021-02-25 07:56:37+00:00,Production,0007 of 0008,G2,None,None,KL8,Sun-SI Delivery,Thursday Morning,iXUsr_Desry(7294),3,1.85,2021-09
37301,2021-02-25 07:16:44+00:00,2021-02-25 07:22:22+00:00,Production,0013 of 0026,A4,None,None,KL2,Sun-SI Delivery,Thursday Morning,LEON JOKOM;,3,5.63,2021-09
32145,2021-02-25 20:03:25+00:00,2021-02-25 20:06:49+00:00,Production,0003 of 0016,R4,None,None,KL14,Sat-NI Delivery,Thursday Afternoon,iXUsr_Meng(10150),4,3.40,2021-09


In [8]:
data_in_use.shape

(64291, 14)